In [1]:
import dask.dataframe as dd
import dask.bag as da
import os
import json
import dask

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

2024-05-22 14:52:01,529 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle ef60af416645fe7e9ea39359d5fed0d9 initialized by task ('shuffle-transfer-ef60af416645fe7e9ea39359d5fed0d9', 8) executed on worker tcp://127.0.0.1:50729
2024-05-22 14:52:12,665 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle ef60af416645fe7e9ea39359d5fed0d9 deactivated due to stimulus 'task-finished-1716382332.660173'


In [2]:
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
import pandas as pd

In [4]:
pokemon_df = pd.read_csv('Files/pokemonDB_dataset.csv')
pokemon_df.head()

,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,...,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
0,Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.2 m (7′03″),135.5 kg (298.7 lbs),"1. Snow Warning, Soundproof (hidden ability)","1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),173,...,273,92,170,311,85,157,295,60,112,240
1,Mega Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.7 m (8′10″),185.0 kg (407.9 lbs),1. Snow Warning,"1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),208,...,339,132,242,399,105,193,339,30,58,174
2,Abra,Psychic,Psi Pokémon,0.9 m (2′11″),19.5 kg (43.0 lbs),"1. Synchronize, 2. Inner Focus, Magic Guard (h...",1 Sp. Atk,"200 (26.1% with PokéBall, full HP)",50 (normal),62,...,141,105,193,339,55,103,229,90,166,306
3,Absol,Dark,Disaster Pokémon,1.2 m (3′11″),47.0 kg (103.6 lbs),"1. Pressure, 2. Super Luck, Justified (hidden ...",2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),163,...,240,75,139,273,60,112,240,75,139,273
4,Mega Absol,Dark,Disaster Pokémon,1.2 m (3′11″),49.0 kg (108.0 lbs),1. Magic Bounce,2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),198,...,240,115,211,361,60,112,240,115,211,361


In [5]:
trainers = dd.read_parquet('Files/trainers_with_pockemon.parquet')
trainers

,first_name,last_name,age,pockemon
npartitions=16,,,,
,string,string,int64,string
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
trainers.head()

,first_name,last_name,age,pockemon
0,Casey,Sheppard,48,Combee
1,Arturo,Blake,108,Chewtle
2,Tamiko,Riddle,113,Wailmer
3,Tamiko,Riddle,113,Impidimp
4,Tamiko,Riddle,113,Natu


In [7]:
trainers_with_pokemon = trainers.merge(pokemon_df, right_on="Pokemon", left_on="pockemon", how="inner")
trainers_with_pokemon

,first_name,last_name,age,pockemon,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,Growth Rate,Egg Groups,Gender,Egg Cycles,HP Base,HP Min,HP Max,Attack Base,Attack Min,Attack Max,Defense Base,Defense Min,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,int64,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
trainers_with_pokemon.dtypes

first_name              string[pyarrow]
last_name               string[pyarrow]
age                               int64
pockemon                string[pyarrow]
Pokemon                 string[pyarrow]
Type                    string[pyarrow]
Species                 string[pyarrow]
Height                  string[pyarrow]
Weight                  string[pyarrow]
Abilities               string[pyarrow]
EV Yield                string[pyarrow]
Catch Rate              string[pyarrow]
Base Friendship         string[pyarrow]
Base Exp                string[pyarrow]
Growth Rate             string[pyarrow]
Egg Groups              string[pyarrow]
Gender                  string[pyarrow]
Egg Cycles              string[pyarrow]
HP Base                           int64
HP Min                            int64
HP Max                            int64
Attack Base                       int64
Attack Min                        int64
Attack Max                        int64
Defense Base                      int64


In [9]:
averge_values_per_team = trainers_with_pokemon.groupby(["first_name", "last_name"]).agg({"Defense Max": "mean", "Attack Max": "mean", "Attack Max": "mean"})
averge_values_per_team.head()

,,Defense Max,Attack Max
first_name,last_name,,
Casey,Sheppard,201.000000,174.000000
Arturo,Blake,218.000000,249.000000
Tamiko,Riddle,245.909091,291.727273
Flo,Conrad,328.000000,284.000000
Donnetta,Owen,267.714286,250.000000


In [10]:
trainers_with_pokemon["bm"] = trainers_with_pokemon["Defense Max"] * trainers_with_pokemon["Attack Max"] * trainers_with_pokemon["Attack Max"]
trainers_with_pokemon["full_name"] = trainers_with_pokemon["first_name"] + trainers_with_pokemon["last_name"]
trainers_with_pokemon.head(20)

,first_name,last_name,age,pockemon,Pokemon,Type,Species,Height,Weight,Abilities,...,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max,bm,full_name
0,Casey,Sheppard,48,Combee,Combee,"Bug, Flying",Tiny Bee Pokémon,0.3 m (1′00″),5.5 kg (12.1 lbs),"1. Honey Gather, Hustle (hidden ability)",...,58,174,42,80,201,70,130,262,6085476,CaseySheppard
1,Arturo,Blake,108,Chewtle,Chewtle,Water,Snapping Pokémon,0.3 m (1′00″),8.5 kg (18.7 lbs),"1. Strong Jaw, 2. Shell Armor, Swift Swim (hid...",...,72,192,38,72,192,44,83,205,13516218,ArturoBlake
2,Tamiko,Riddle,113,Wailmer,Wailmer,Water,Ball Whale Pokémon,2.0 m (6′07″),130.0 kg (286.6 lbs),"1. Water Veil, 2. Oblivious, Pressure (hidden ...",...,130,262,35,67,185,60,112,240,12699140,TamikoRiddle
3,Tamiko,Riddle,113,Impidimp,Impidimp,"Dark, Fairy",Wily Pokémon,0.4 m (1′04″),5.5 kg (12.1 lbs),"1. Prankster, 2. Frisk, Pickpocket (hidden abi...",...,103,229,40,76,196,50,94,218,7455726,TamikoRiddle
4,Tamiko,Riddle,113,Natu,Natu,"Psychic, Flying",Tiny Bird Pokémon,0.2 m (0′08″),2.0 kg (4.4 lbs),"1. Synchronize, 2. Early Bird, Magic Bounce (h...",...,130,262,45,85,207,70,130,262,9837468,TamikoRiddle
5,Tamiko,Riddle,113,Audino,Audino,Normal,Hearing Pokémon,1.1 m (3′07″),31.0 kg (68.3 lbs),"1. Healer, 2. Regenerator, Klutz (hidden ability)",...,112,240,86,159,298,50,94,218,17164800,TamikoRiddle
6,Tamiko,Riddle,113,Chimchar,Chimchar,Fire,Chimp Pokémon,0.5 m (1′08″),6.2 kg (13.7 lbs),"1. Blaze, Iron Fist (hidden ability)",...,108,236,44,83,205,61,114,243,11417680,TamikoRiddle
7,Tamiko,Riddle,113,Escavalier,Escavalier,"Bug, Steel",Cavalry Pokémon,1.0 m (3′03″),33.0 kg (72.8 lbs),"1. Swarm, 2. Shell Armor, Overcoat (hidden abi...",...,112,240,105,193,339,20,40,152,55604475,TamikoRiddle
8,Tamiko,Riddle,113,Meganium,Meganium,Grass,Herb Pokémon,1.8 m (5′11″),100.5 kg (221.6 lbs),"1. Overgrow, Leaf Guard (hidden ability)",...,153,291,100,184,328,80,148,284,27394888,TamikoRiddle
9,Tamiko,Riddle,113,Trapinch,Trapinch,Ground,Ant Pit Pokémon,0.7 m (2′04″),15.0 kg (33.1 lbs),"1. Hyper Cutter, 2. Arena Trap, Sheer Force (h...",...,85,207,45,85,207,10,22,130,22269888,TamikoRiddle


In [11]:
sorted_df = trainers_with_pokemon.sort_values(["first_name", "last_name", "bm"], ascending=False)
sorted_df

,first_name,last_name,age,pockemon,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,Growth Rate,Egg Groups,Gender,Egg Cycles,HP Base,HP Min,HP Max,Attack Base,Attack Min,Attack Max,Defense Base,Defense Min,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max,bm,full_name
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,int64,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
best_pk = sorted_df.groupby(["first_name", "last_name"]).agg({"Pokemon": "first", "HP Max": "first", "Attack Max": "first", "Defense Max": "first"}).compute()
best_pk.head()


2024-05-22 14:52:14,383 - distributed.worker.memory - WARNING - Worker is at 85% memory usage. Pausing worker.  Process memory: 1.40 GiB -- Worker memory limit: 1.64 GiB
2024-05-22 14:52:14,383 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.40 GiB -- Worker memory limit: 1.64 GiB
2024-05-22 14:52:14,402 - distributed.worker.memory - WARNING - Worker is at 78% memory usage. Resuming worker. Process memory: 1.28 GiB -- Worker memory limit: 1.64 GiB


Pokemon  HP Max  Attack Max  Defense Max
first_name last_name                                                  
Zulma      York             Rampardos     398         471          240
           Woods      Hisuian Avalugg     394         388          513
           Wilder            Minccino     314         218          196
           Whitley        Stonjourner     404         383          405
           Wells           Sandaconda     348         344          383

In [13]:
best_pk.reset_index(inplace=True)

In [14]:
best_pk.loc[(best_pk["first_name"] == "Tamiko") & (best_pk["last_name"] == "Riddle")]

,first_name,last_name,Pokemon,HP Max,Attack Max,Defense Max
76255,Tamiko,Riddle,Escavalier,344,405,339


# Storing Data on WS

In [15]:
averge_values_per_team_df = averge_values_per_team.compute().reset_index()
averge_values_per_team_df.head()

,first_name,last_name,Defense Max,Attack Max
0,Casey,Sheppard,201.000000,174.000000
1,Arturo,Blake,218.000000,249.000000
2,Tamiko,Riddle,245.909091,291.727273
3,Flo,Conrad,328.000000,284.000000
4,Donnetta,Owen,267.714286,250.000000


In [18]:
import requests

In [19]:
for row in averge_values_per_team_df.head(1000).itertuples():
    response = requests.post("http://localhost:8000/trainer", json={
        
          "first_name": row[1],
          "last_name": row[2],
          "avg_defense": row[3],
          "avg_attack": row[4]

    })

    if response.status_code != 200:
        print(response.status_code, response.text)
        break


In [17]:
for row in best_pk.itertuples():
    # TODO save values on WS

SyntaxError: incomplete input (3269177123.py, line 2)